In [5]:
from sympy.combinatorics import *
from itertools import combinations
from sympy.interactive import init_printing

from tqdm.notebook import tqdm

from math import factorial
import numpy as np

import pickle

init_printing(perm_cyclic=True, pretty_print=False) #prints permutations in compact notation
perm_type = permutations.Permutation

NameError: name 'permutations' is not defined

# Reproduce (correct) coefficients from project catalog

In [6]:
def gen_ctype(l: dict,n: int):
    '''
    Input:
        l - cycle decompostion as a dictionary including 1 cycles. e.g. {1:2, 2:1} is a transposition in S_4
        n - specifies order of symmetric group
    Output:
        Generates all permutations of specified cycletype as a list
    '''
    
    has_to_be_partition = 0
    for key, value in l.items():
        has_to_be_partition = has_to_be_partition + key*value
    if has_to_be_partition < n:
        print("Invalid cycle type format.")
    else:
        list_with_cycle_type = []
        
        #run through all permutations and append them if they have the correct cycletype
        for i in list(symmetric(n)): 
            if i.cycle_structure == l:
                list_with_cycle_type.append(i)            
        return list_with_cycle_type
    
g2 = gen_ctype({1:2,2:1},4) #An example of all permutations with cycletype 2 (transpositions) in S_4
g2

[(2 3), (3)(1 2), (1 3), (3)(0 1), (3)(0 2), (0 3)]

In [7]:
def norm(p, cycle_dict = None):
    '''
    Input: permutation p, optionally the cycle structure of p.
    Output: The norm of p
    '''
    #If the cycle type of p is already computed we can reuse the result.
    if cycle_dict == None:
        cycle_dict = p.cycle_structure
    sum_of_lengths = 0
    num_cycles = 0
    
    #We use the formula N(sigma) = d-l if sigma has cyclestructure C_1C_2...C_l 
    for key, value in cycle_dict.items():
        sum_of_lengths += key*value
        num_cycles += value
    return sum_of_lengths-num_cycles

In [9]:
def norm_check(p,q, norm_sum = None):
    '''
    Input: Permutations p and q, optionally the sum of their norms.
    Output: True if N(pq) = N(p) + N(q), otherwise False.
    '''
    if norm_sum == None:
        norm_sum = norm(p) + norm(q)
    p_trans = set(p.transpositions())
    q_trans = set(q.transpositions())
    
    if p_trans.intersection(q_trans) == set([]):
        return norm(p*q)==norm_sum
    return False

In [10]:
def Aprod(p_list, q_list, norm_sum = None):
    '''
    Input:
        - p_list and q_list, both lists of permutations corresponding to a conjugacy class in S_n
        - The sum of the norm of the conjugacy classes (optional)
    Output:
        - The product of the two conjugacy classes according to A(d)
    '''
    if norm_sum:
        return [s*t for s in p_list for t in q_list if norm_check(s,t,norm_sum)]
    return [s*t for s in p_list for t in q_list if norm_check(s,t)]

# Generate powers of g2, as 
g2_square = Aprod(g2,g2,2)
g2_cube = Aprod(g2,g2_square,3)
g2_square[0:5]

[(1 2 3), (1 3 2), (0 1)(2 3), (0 2 3), (0 3 2)]

In [13]:
def ctype(p):
    '''
    Input:
        Sympy permutation
    Output:
        Gives cycletype of permutation as a tuple with i'th coordinate being multiplicity of i-cycles.
    '''
    d = p.cycle_structure
    max_val = max(d.keys())
    L = []
    for i in range(1,max_val+1):
        if i in list(d.keys()):
            L.append(d[i])
        else:
            L.append(0)
    return tuple(L)

ctype(Permutation(3)(1,3)) # has 2 1-cycles (0 and 2) and one 2-cycle

(2, 1)

In [14]:
def decompose(plist: list, progressbar=True):
    '''
    Input: List of permutations
    Output: Decomposes list by occurences of cycletype
    '''
    cycle_count = {}
    for p in tqdm(plist, disable = progressbar):
        p_type = ctype(p)
        if p_type in list(cycle_count.keys()):
            cycle_count[p_type] += 1
        else:
            cycle_count[p_type] = 1
    return cycle_count
#Decomposes list of permutations into representatives of the conjugacy class, e.g. (g_2)^2 has 24 3-cycles and 6 2-cycles
print("g2 squared decomp", decompose(g2_square),
      "\n" "g2 cubed decomp", decompose(g2_cube))

g2 squared decomp {(1, 0, 1): 24, (0, 2): 6} 
g2 cubed decomp {(0, 0, 0, 1): 96}


In [10]:
def num_ctype(ctuple):
    '''
    Input: cycle type as a tuple
    Output: number of permutations with given cycle type using 
    '''
    n = sum((i+1)*ctuple[i] for i in range(len(ctuple)))
    numerator = factorial(n)
    denominator = 1
    for i in range(len(ctuple)):
        denominator *= ((i+1)**ctuple[i]) * factorial(ctuple[i])
    return numerator // denominator

num_ctype((2,1)) #Size of the conjugacy class of 2-cycles in d = 2*1+1*2 = 4

6

In [11]:
def to_string(d):
    if type(d) == dict:
        s = ""
        for key, value in d.items():
            s += str(value) + "*" + "g_"+ to_string(key) + "  +  "
        return s[0:-3]
    s = ""
    for i,k in enumerate(d):
        if i!=0 and k!= 0:
            s+= str(i+1) + "^" + str(k) + ","
    return s[0:-1]

In [12]:
def decompose2(plist, string_output = False):
    '''
    Input: list of permutations
    Output: decomposition according to A(d)
    '''
    d = decompose(plist)
    for i in d.keys():
        d[i] //= num_ctype(i)
        
    if string_output:
        return to_string(d)
    return d
#Same as decompose, but divided by the size of the conjugacy class
print("g2 square", decompose2(g2_square), 
      "\n" "g2 cube", decompose2(g2_cube))

### Try powers of g2 in higher d

In [ ]:
d = 6
g2 = gen_ctype({1:(d-2), 2:1}, d)

g2_powers = [Aprod(g2,g2,2)]

for i in tqdm(range(3,3+3)):
    g2_powers.append(Aprod(g2,g2_powers[-1],i))

In [ ]:
g2_powers_decomp = []
for i in tqdm(g2_powers):
    g2_powers_decomp.append(decompose2(i))
    print(g2_powers_decomp[-1])

# Relations

In [16]:
def gen_all(d):
    ''' Input: non-negative interger
        Output: Dictionary of elements in the symmetric group with key being cycle type and value all permutations in S(d) with
        that cycle type.
    '''
    answer = {}
    
    for i in tqdm(symmetric(d), total = factorial(d)):
        C = ctype(i)
        if C in answer.keys():
            (answer[C]).append(i)
        else:
            answer[C] = [i]
    answer.pop((d,),None)
    return answer

def norm2(p_tuple):
    #Computes norm from cyclestructure written as a tuple
    ans = 0
    for i,p in enumerate(p_tuple):
        ans += i*p
    return ans

In [ ]:
d=8
basis = gen_all(d)

In [ ]:
#Give multiplication table of generators
generators = [basis[(d-2*i,i)] for i in range(1,d//2+1)]
print("Sample permutations from generators:", [g[0] for g in generators])
for i,g1 in enumerate(generators):
    for j,g2 in enumerate(generators[i:]):
        print("basis:",i+1,j+i+1,"Product:",decompose2(Aprod(g1,g2,2*i+j+2),True))

In [ ]:
#Multiplication table of g2 with other elements
g2_mult_table = []

for k,v in tqdm(basis.items()):
    s ="g_2 * g_" + to_string(k) + " = "
    s += decompose2(Aprod(basis[(d-2,1)],v, norm2((d-2,1)) + norm2(k)), string_output = True)
    g2_mult_table.append(s)
    print(s)

In [ ]:
#Entire multiplication table
mult_table = []

for k1,v1 in tqdm(basis.items()):
    for k2,v2 in basis.items():
        if k1 >= k2:
            n = norm2(k1) + norm2(k2)
            if n < d:
                s = "g_"+to_string(k1) + " * g_" + to_string(k2)
                s += " = " + decompose2(Aprod(v1,v2,n), string_output = True)
                mult_table.append(s)
                print(s)

### Save result as a pickle file

In [ ]:
#with open('g2table.pkl', 'wb') as f:
#    pickle.dump(g2_mult_table, f)

In [ ]:
#with open('prod_obs.pkl', 'wb') as f:
#    pickle.dump(products, f)

In [ ]:
with open('g2table.pkl', 'rb') as f:
    products = pickle.load(f)
for line in products:
    print(line[0:-3])

In [ ]:
for line in products:
    print(line[0],line[1], list(line[2].values()))

# Relations 2.0

In [14]:
f = lambda u, v: Aprod(u,v)
g = lambda u: decompose2(u, string_output = True)

def gen_monoms(d,generators = [],monomials = [], alphabet = "xyzwæøå"):
    
    if generators == []:
        basis = gen_all(d)
        generators = [basis[(d-2-2*i, i+1)] for i in range(d//2)]
    
    if monomials == []:
        monomials.append((0,[Permutation(d-1)()], ""))
    
    mon = (norm((generators[0])[0]), generators[0], alphabet[0])

    for m in monomials:
        if m[0] + mon[0] < d:
            product = f(mon[1],m[1])
            monomials.append((m[0] + mon[0], product, m[2]+mon[2]))
            print(m[2]+mon[2])
    
    if len(generators)==1:
        return monomials
    else:
        return gen_monoms(d, generators[1:], monomials, alphabet[1:])


            
def rel_experiment(d):
    monoms = gen_monoms(d)
    for m in monoms:
        if len(m[2])>1:
            print("monomial:", m[2], "decomp:", g(m[1]))

In [17]:
rel_experiment(6)


x
xx
xxx
xxxx
xxxxx
y
xy
xxy
xxxy
yy
xyy
z
xz
xxz
yz
monomial: xx decomp: 3*g_3^1  +  2*g_2^2  
monomial: xxx decomp: 16*g_4^1  +  9*g_2^1,3^1  +  6*g_2^3  
monomial: xxxx decomp: 125*g_5^1  +  64*g_2^1,4^1  +  54*g_3^2  
monomial: xxxxx decomp: 1296*g_6^1  
monomial: xy decomp: 2*g_4^1  +  3*g_2^1,3^1  +  3*g_2^3  
monomial: xxy decomp: 25*g_5^1  +  20*g_2^1,4^1  +  18*g_3^2  
monomial: xxxy decomp: 324*g_6^1  
monomial: yy decomp: 5*g_5^1  +  4*g_2^1,4^1  +  9*g_3^2  
monomial: xyy decomp: 81*g_6^1  
monomial: xz decomp: 2*g_2^1,4^1  
monomial: xxz decomp: 12*g_6^1  
monomial: yz decomp: 3*g_6^1  


# Trashcan

## Output coefficient matrix

In [ ]:
all_keys = set([])
for decomp in g2_powers_decomp:
    all_keys = all_keys.union(set(decomp.keys()))
all_keys = all_keys - {(d-2, 1), (d,)}
all_keys = list(all_keys)
all_keys.sort()
all_keys

In [ ]:
(n,m) = (len(g2_powers_decomp),len(all_keys))
A = np.zeros((n,m)).astype(int)

for i in range(n):
    for j in range(m):
        try:
            A[i,j] = (g2_powers_decomp[i])[all_keys[j]]
        except:
            pass

A

In [ ]:
def test_ctype(p: perm_type, D):
    '''
    Input:
        p
    '''
    d = p.cycle_structure
    d.pop(1, None)
    if d.keys() == D.keys():
        for key in D.keys():
            if d[key] != D[key]:
                return False
        return True
    return False

In [ ]:
products = []
for i in range(1,d//2+1):
    p = basis[(d-2*i, i)]
    for j in range(i,d//2+1):
        q = basis[(d-2*j, j)]
        dic = decompose2(Aprod(p,q, i + j))
        products.append([i,j, dic])
        print(i,j, to_string(dic))

In [ ]:
def memoize(f):
    memo = {}
    def helper(x):
        if x not in memo:            
            memo[x] = f(x)
        return memo[x]
    return helper

In [ ]:
def norm_check(p,q, norm_sum = None):
    '''
    
    '''
    if norm_sum == None:
        norm_sum = norm(p) + norm(q)
    return norm(p*q)==norm_sum

In [ ]:
d=6
basis = gen_all(d)

In [ ]:
x = basis[(d-2,1)]
y = basis[(d-4,2)]
z = basis[(d-6,3)]

In [ ]:
f = lambda u, v: Aprod(u,v)
g = lambda u: print(decompose2(u, string_output = True))


print("w = 2")
x2 = f(x,x)
g(x2)

print("w = 3")
x3 = f(x2,x)
g(x3)

xy = f(x,y)
g(xy)

print("w = 4")
x4 = f(x3,x)
g(x4)

x2y = f(x,xy)
g(x2y)

y2 = f(y,y)
g(y2)

xz = f(x,z)
g(xz)

print("w = 5")
g(f(x4,x))
g(f(x3,y))
g(f(x,y2))
g(f(x,xz))
g(f(y,z))
